# Integrating bluetooth and "with who" questions from timediaries to establish the number of people around in each moment and place
The bluetooth data have been cleaned from devices non identifying people, thus they can be used to detect the number of people around in every record, encoded as 0, between 1 and 3, and 4 or more.
In case the bluetooth data were not sufficient to cover all the records, the "with who" questions from the time diaries were used to fill the missing values.

In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [ ]:
df = pd.read_csv('with_who_map.csv')
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

In [ ]:
# For other columns with missing values, fill them 
df = df.fillna('unknown')

# Create bands for variables that we want to use in the model
import ast

for row in range(len(df)):
    if df.bluetooth[row] != 'unknown':
        lst = ast.literal_eval(df.bluetooth[row])
        if len(lst)<3 and len(lst)>0:
            df.at[row, 'Num_of_people'] = '1 - 3'
        elif len(lst)==0:
            df.at[row, 'Num_of_people'] = '0'
        else:
            df.at[row, 'Num_of_people'] = '>= 4'
    else:
        if df.with_who[row] == 'Alone':
            df.at[row, 'Num_of_people'] = '0'
        elif df.with_who[row] == 'Roommate(s)':
            df.at[row, 'Num_of_people'] = '1 - 3'
        else:
            df.at[row, 'Num_of_people'] = 'unknown'

In [ ]:
# given the activity and the corresponding place category, put the class that is more likely to be true
for row in range(len(df)):
    if df.bluetooth[row] == 'unknown':
        # family
        if df['with_who'][row] == 'Relative(s)':
            df.at[row, 'Num_of_people'] = '1 - 3'
        
        # lecture
        elif df.what[row] == 'Lecture/seminar/conference/university meeting' and df['with_who'][row] == 'Classmate(s)':
            df.at[row, 'Num_of_people'] = '>= 4'
        elif df.what[row] == 'Free time/hobbies' and df['with_who'][row] == 'Classmate(s)':
            df.at[row, 'Num_of_people'] = '1 - 3'
            
        # room
        elif 'Home apartment /room' in df.places_dict[row] and df['with_who'][row] == 'Friend(s)':
            df.at[row, 'Num_of_people'] = '1 - 3'
        
        # social
        elif df.what[row] == 'Social life (Socialising, visiting, receiving, conversating with family, relatives, friends, classmate, visitors, neighbour, and others)' \
        or df.what[row] == 'Happy Hour/Drinking/Party' and df['with_who'][row] == 'Friend(s)':
            df.at[row, 'Num_of_people'] = '>= 4'
            
        # when shopping
        elif df.what[row] == 'Other Shopping' or df.what[row] == 'Grocery Shopping' and df['with_who'][row] == 'Friend(s)': 
            df.at[row, 'Num_of_people'] = '1 - 3'
            
        # sport
        elif (df.what[row] == 'Walking' or df.what[row].startswith('Sport')) and (df['with_who'][row] == 'Friend(s)' or df['with_who'][row] == 'Classmate(s)'): 
            df.at[row, 'Num_of_people'] = '1 - 3'

In [ ]:
df[df.Num_of_people=='unknown']

In [ ]:
# mood function to associate emoticon to value of avg.mood
def mood_function(mood):
    if mood == 'unknown':
        return "<b>?</b>"
    if float(mood) < 2.0: # sad & very sad
        return "<img src='https://cdn-icons-png.flaticon.com/512/927/927561.png' width='20'/>"
    elif float(mood) >= 2 and float(mood) < 2.6: # neutral
        return "<img src='https://cdn-icons-png.flaticon.com/512/927/927557.png' width='20'/>"
    else: # >= 3 --> happy & very happy
        return "<img src='https://cdn-icons-png.flaticon.com/512/1023/1023656.png' width='20'/>"
    
# image people around
def num_ppl(num):
    if num == '0':
        return "<img src='images_ppl/alone.png' width='40'/>"
    elif num == '1 - 3':
        return "<img src='images_ppl/small-group.png' width='70'/>"
    elif num == '>= 4':
        return "<img src='images_ppl/big-group.png' width='80'/>"

    
# color markers by number of people around
def color_marker(n): 
    if n=='0':
        return 'red'
    elif n=='1 - 3':
        return 'orange'
    else: # n >= 4
        return 'green'

In [ ]:
# cut address
for row in range(len(df)):
    lst = df.City[row].split(',')
    # address
    s = lst[0] + ',' + lst[1]  
    # adding city
    if lst[-6] not in s:
        s = s + ',' + lst[-6]
    elif lst[-5] not in s:
        s = s + ',' + lst[-5]
    # change value in df
    df.at[row, 'City'] = s

In [ ]:
import ast
# max(ast.literal_eval(df.places_dict[4]), key=ast.literal_eval(df.places_dict[4]).get)

for row in range(len(df)):
    if df.places_dict[row]!='{}':
        df.at[row, 'place'] = max(ast.literal_eval(df.places_dict[row]), key=ast.literal_eval(df.places_dict[row]).get)
    else:
        df.at[row, 'place'] = 'unknown'

In [ ]:
# legend
import branca

legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    top: 20px;
    right: 50px;
    width: 250px;
    height: 110px;
    z-index:9999;
    font-size:14px;
    ">
    <p style="color:black;font-size:100%;margin-left:10px;">With who?<br>(Bluetooth & time diaries)</p>
    <p><a style="color:red;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;Alone</p>
    <p><a style="color:orange;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;Between 1 and 3 other people</p>
    <p><a style="color:green;font-size:150%;margin-left:20px;">&#9632;</a>&emsp;4 people or more</p>
</div>
<div style="
    position: fixed; 
    top: 10px;
    right: 50px;
    width: 250px;
    height: 180px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;

    opacity: 0.7;
    ">
</div>
{% endmacro %}
'''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)

In [ ]:
# center the map around north-east of Italy
# box = [45.65, 11.75] # lat, long
# zoom 
box = [46.07, 11.135]

# create the map
my_map = folium.Map(location = box, zoom_start = 15)


for row in range(len(df)): 
    pop_up = folium.Html(mood_function(df.mood[row]) + '<br>' + '<strong>Address: </strong>' + str(df.City[row]) + 
                         '<br> <strong>Subjective place: </strong>' + str(df.place[row]) + 
                         '<br><br><strong>' + num_ppl(df.Num_of_people[row]) + '<br>',
                         #'Activity:</strong>' + '<br>' + str(df.what[row]), 
                         script=True)
    popup = folium.Popup(pop_up, max_width=2650)
    folium.Marker([df.lat[row], df.long[row]], popup = popup,
                 icon=folium.Icon(color = color_marker(df.Num_of_people[row]), icon_color = 'white', icon='bluetooth-b', prefix='fa')).add_to(my_map)

my_map.get_root().add_child(legend) # add legend
my_map


In [ ]:
my_map.save('bluetooth_map.html')